# Learning Bitmaths for Bitmaps

## Introduction

Bitmaps are used to index data compactly. This can be extremely useful when building blockchain applications in resource constrained environments. A notable use case pioneered by Uniswap V3 is for concentrated liquidity AMMs which need to often and efficiently query the next chunk of available liquidity to swap through.

A bitmap is made up of **words** which are chunks of binary strings. We query each word by passing it's index into a map from. We need to chunk out the map into separate words because because a single integer can only hold a finite number of bits (*note: Python ints are 32 bits long so some details may change depending on the language you're using, but the general logic is still the same.*)

In [ ]:
#Initializing the bitmap and 3 words
bitmap = {} 
bitmap[0] = 0
bitmap[1] = 0
bitmap[2] = 0

The first bit trick used in the bitmap is the equivalence of >> x and floor division by 2 ** x. This is because for every bit you shift right, you floor divide the number by 2. So by doing it x times, we get floor division by 2 ** x. 

This is used to figure out which word to check for a given tick since it perfectly segments the global tick indices into the number of words there are i.e., if we the max number of ticks is 100, and each word is 32 bits, then we need 4 words to house all of the tick data (1 -> 0-31, 2 -> 32-63, 3 -> 64-95, 4 -> 96-99). 

In [ ]:
assert int(21 / 32) == 21 >> 5 == 0
print(int(21 / 32), 21 >> 5, 0)

assert int(61 / 32) == 61 >> 5 == 1
print(int(61 / 32), 61 >> 5, 2)

assert int(66 / 32) == 66 >> 5 == 2
print(int(66 / 32), 66 >> 5, 2)

We also use modulo division to figure out what the "local index" is of tick i.e., the index within a particular word.

In [ ]:
assert 21 % 32 == 21
print(21 % 32, 21)

assert 61 % 32 == 29
print(61 % 32, 29)

assert 66 % 32 == 2
print(66 % 32, 2)

Let's put them together to create a function which returns the position.

In [ ]:
def position(currentTick: int) -> tuple(int, int):
    # Remember that >> 5 is equivalent to floor division by 2**5
    wordIndex = currentTick >> 5

    # Used to determine the "local index" of the tick within the word.
    localTickIndex = currentTick % 32
    return wordIndex, localTickIndex 


The next bit trick used is the creation of a mask that can be used to flip ticks on and off. This trick uses the fact that 1 is represented in binary as "0...001" . Thus by bitshifting it leftward by x bits, we ensure that there is now a 1, x places to the left i.e., index (*note: indexing is left to right in binary*).

This max will be used in bitwise XOR operations in the future to turn ticks on (if they're off and are being initialized - first liquidity is being deposited), or off (if they're on and being unitialized - all liquidity is being withdrawn).

In [ ]:
# 1 == "0...001"
binaryString = 1
print("{0:b}".format(binaryString))

#1 << 5 == "0...0100000"
binaryString = binaryString << 5
print("{0:b}".format(binaryString))

# 1 << 3 == "0...01000"
# 1 << 5 ^ 1 << 3 == "0...0101000"
binaryString ^= 1 << 3
print("{0:b}".format(binaryString))


Exclusive or, XOR for short, is a logical operation which evaluates to true if only one of the arguments is true. I.e., 1 ^ 0 == 0 ^ 1 == 1 and 0 ^ 0 == 1 ^ 1 == 0 (*note: in most languages XOR is called by using ^ or ^= if we want to apply it and store it to the same string*).

Because we use XOR, before flipping a bit on we must check that it has not already been initialized. If we do not check this, we risk uninitializing ticks that should remain initialized. This can be done dynamically when updating the state to increment liquidity. For now let's assume that this check was already done.

In [ ]:
# 1 == "0...001"
binaryString = 1
print("{0:b}".format(binaryString))

#1 << 5 == "0...0100000"
binaryString = binaryString << 5
print("{0:b}".format(binaryString))

#Turning a bit on at index 3
# 1 << 3 == "0...01000"
# 1 << 5 ^= 1 << 3 == "0...0101000"
binaryString ^= 1 << 3
print("{0:b}".format(binaryString))

# Turning a bit off at index 3
# binaryString ^= 1 << 3 == "0...0100000"
binaryString ^= 1 << 3
print("{0:b}".format(binaryString))
